# Testing RF6 Impact Attestation Metrics

Spec [here](https://plaid-cement-e44.notion.site/Impact-Attestations-Data-b8b1c79a624c46ad94305c74def64783)

1. Number of Attestations (only want the count from Citizens and Top Delegates)
2. Number of Attestations by Citizens
3. Number of Attestations by Top delegates
4. Average NPS score of Citizens and Top delegates
5. Most positive superlative
    1. At least 20 citizen or top delegates reviews
    2. Over 95% of those reviews feel “extremely upset”
    3. Over 95% of those reviews rate very likely to recommend (9-10)
6. "Can't live without" superlative
    1. At least 20 citizen or top delegate reviews
    2. Over 90% of those reviews feel “extremely upset”
7. Percentage distribution of different ratings by citizens and top delegates [Extremely upset, Somewhat upset, Neutral]
    1. Percentage distribution of different ratings by citizens
    2. Percentage distribution of different ratings by delegates
8. Number of elected governance members that attested
    1. Number of each group of elected governance members that attested
    2. Average NPS score per elected governance group
    3. Average feeling per elected governance group

In [1]:
import datetime
import json
import numpy as np
import pandas as pd
import random
import secrets

# Load relevant attestations

In [2]:
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

In [3]:
project_attestation_data = load_json('data/RetroFunding_ApplicationAttestations.json')
print("Project attestations:", len(project_attestation_data))

badgeholder_attestation_data = load_json('data/RetroFunding_BadgeholderAttestations.json')
print("Badgeholder attestations:", len(badgeholder_attestation_data))

gov_council_attestation_data = load_json('data/Governance_CouncilAttestations.json')
print("Gov Council attestations:", len(gov_council_attestation_data))

impact_attestation_data = load_json('data/MGL_ImpactAttestations.json')
print("Impact attestations:", len(impact_attestation_data))

Project attestations: 975
Badgeholder attestations: 713
Gov Council attestations: 86
Impact attestations: 188


# Clean / filter attestation data

In [4]:
relevant_project_attestations = []
for a in project_attestation_data:
    if a['attester'] != '0xF6872D315CC2E1AfF6abae5dd814fd54755fE97C':
        continue
    if int(a['round']) < 5:
        continue
    relevant_project_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'projectRefUID': a['projectRefUID'].lower(),
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated'])
    })
        
df_projects = pd.DataFrame(relevant_project_attestations)        
print("Relevant project attestations:", len(relevant_project_attestations))        

Relevant project attestations: 362


In [5]:
relevant_badgeholder_attestations = []
for a in badgeholder_attestation_data:
    if a['attester'] != '0x621477dBA416E12df7FF0d48E14c4D20DC85D7D9':
        continue
    relevant_badgeholder_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'rpgfRound': a['rpgfRound'],
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated'])
    })
print("Relevant badgeholder attestations:", len(relevant_badgeholder_attestations))            

Relevant badgeholder attestations: 216


In [6]:
relevant_gov_council_attestations = []
for a in gov_council_attestation_data:
    if a['attester'] != '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F':
        continue
    if int(a['govSeason']) < 5:
        continue
    relevant_gov_council_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'govRole': a['govRole'],
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated'])
    })
print("Relevant gov council attestations:", len(relevant_gov_council_attestations))            

Relevant gov council attestations: 43


In [7]:
relevant_impact_attestations = []
for a in impact_attestation_data:
    qs = a.get('metadata', {}).get('impactAttestations', [])
    
    if len(qs) < 2:
        continue
    
    nps = pmf = None
    for q in qs:
        val = q.get('value')
        if not val:
            continue
        if q['name'] == 'Likely to Recommend':
            nps = val
        elif q['name'] == 'Feeling if didnt exist':
            pmf = val
    
    if not (nps or pmf):
        continue
        
    relevant_impact_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'farcasterID': a['farcasterID'],
        'projectRegUID': a['projectRegUID'].lower(),
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated']),
        'issuer': a['issuer'],
        'metadataurl': a['metadataurl'],
        'nps_score': pd.to_numeric(nps),
        'pmf_score': pd.to_numeric(pmf)
    })
    
print("Relevant impact attestations:", len(relevant_impact_attestations))

Relevant impact attestations: 36


In [8]:
df_attestations_actual = pd.DataFrame(relevant_impact_attestations)
df_attestations_actual.tail(1).T

,35
id,0xfa0eae3eacef3911aab96912f012db75443fa7a99d50...
attester,0x7484aabfef9f39464f332e632047983b67571c0a
recipient,0x4a6894dd556fab996f8d50b521f900caeedc168e
farcasterID,"{'type': 'BigNumber', 'hex': '0x050fa9'}"
projectRegUID,0x51504348243d8e8fa59cf2ba983d9255373e5eb23fe2...
timeCreated,2024-10-10 04:10:35
issuer,MGL
metadataurl,https://gateway.pinata.cloud/ipfs/QmRod9xTy6fR...
nps_score,8.0
pmf_score,3.0


# Determine constants

In [9]:
PROJECTS = [a['projectRefUID'] for a in relevant_project_attestations]
len(PROJECTS)

362

In [10]:
COUNCILS = ['Anticapture Commission', 'Code of Conduct Council', 'Collective Feedback Committee',
            'Developer Advisory Board', 'Grants Council', 'Security Council']

COUNCILS = list(pd.DataFrame(relevant_gov_council_attestations)['govRole'].unique())
COUNCILS

['Optimism Grants Council Member',
 'Anticapture Commission Member',
 'Developer Advisory Board Member',
 'Code of Conduct Council Member',
 'Code of Conduct Council',
 'ACC (Second half)']

In [11]:
DELEGATES = {a['recipient']: set() for a in relevant_gov_council_attestations}
for a in relevant_gov_council_attestations:
    DELEGATES[a['recipient']].add(a['govRole'])
DELEGATES

{'0x4f41877773e44f2275da1942fee898556821bf66': {'Optimism Grants Council Member'},
 '0x94db037207f6fb697dbd33524aadffd108819dc8': {'Anticapture Commission Member',
  'Optimism Grants Council Member'},
 '0x6eda5acaff7f5964e1ecc3fd61c62570c186ca0c': {'Anticapture Commission Member',
  'Optimism Grants Council Member'},
 '0x75536cf4f01c2bfa528f5c74ddc1232db3af3ee5': {'Anticapture Commission Member',
  'Optimism Grants Council Member'},
 '0xf68d2bfcecd7895bba05a7451dd09a1749026454': {'Optimism Grants Council Member'},
 '0xd2e351099971bfdbb484d000549776fae7e1f889': {'Developer Advisory Board Member'},
 '0x7899d9b1181cbb427b0b1be0684c096c260f7474': {'Anticapture Commission Member'},
 '0x1f9e7984eddb135ab6d6f354e229970bb332dcc4': {'Anticapture Commission Member',
  'Code of Conduct Council Member'},
 '0xf346100e892553dceb41a927fb668da7b0b7c964': {'Developer Advisory Board Member'},
 '0xa142ab9eab9264807a41f0e5cbdab877d204e233': {'Anticapture Commission Member',
  'Optimism Grants Council Memb

In [12]:
BADGEHOLDERS = set([a['recipient'] for a in relevant_badgeholder_attestations])
len(BADGEHOLDERS)

149

In [13]:
ATTESTERS = {}
for addr in BADGEHOLDERS:
    if addr not in ATTESTERS:
        ATTESTERS.update({
            addr: {
                'is_citizen': True,
                'is_top_delegate': False,
                'governance_membership': []                
            }
        })
for (addr,membership) in DELEGATES.items():
    if addr not in ATTESTERS:
        ATTESTERS.update({
            addr: {
                'is_citizen': False,
                'is_top_delegate': True,
                'governance_membership': list(membership)
            }
        })
    else:
        ATTESTERS[addr]['is_top_delegate'] = True
        ATTESTERS[addr]['governance_membership'] = list(membership)


# Create some dummy data attestations

In [14]:
NUM_PROJECTS = 30
NUM_ATTESTERS = 100
NUM_ATTESTATIONS = 1000
PMF_SCALE = 5
NPS_SCALE = 10


projects = {
    p: random.random()
    for p in random.choices(PROJECTS, k=NUM_PROJECTS)
}

attesters = {
    addr:ATTESTERS[addr]
    for addr in random.choices(list(ATTESTERS.keys()), k=NUM_ATTESTERS)
}

attestations = []
for i in range(NUM_ATTESTATIONS):
    random_project = random.choice(list(projects.keys()))
    random_attester = random.choice(list(attesters.keys()))
    project_impact = projects.get(random_project)
    pmf_score = random.randint(int(project_impact*PMF_SCALE), PMF_SCALE)
    nps_score = random.randint(int(project_impact*NPS_SCALE), NPS_SCALE)
    attestations.append({
        'uid': f"0x{secrets.token_hex(32)}",
        'attester': random_attester,
        'rf6_applicationUID': random_project,
        'pmf_score': max(1,pmf_score),
        'nps_score': max(1,nps_score),
        **attesters[random_attester]
    })

In [15]:
with open("data/dummy_attestations.json", "w") as f:
    json.dump(attestations,f,indent=2)

attestations[0]

{'uid': '0xf4d4b6d3df0976b80c29b6dc2311c0aa50caf04035d2d54f2ae36b9d78ff266d',
 'attester': '0x665d84fffddd72d24df555e6b065b833478dffca',
 'rf6_applicationUID': '0x7333cac46b20999de07f4d7be5146c10a30f19586907bd2ce986b81af340cb97',
 'pmf_score': 1,
 'nps_score': 5,
 'is_citizen': True,
 'is_top_delegate': False,
 'governance_membership': []}

# Implement a few of the metrics

In [16]:
df_sim = pd.DataFrame(attestations)
df_sim = df_sim[df_sim.is_citizen | df_sim.is_top_delegate]

In [17]:
def most_positive_superlative(rf6_app_uid):
    dff_sim = df_sim[df_sim['rf6_applicationUID'] == rf6_app_uid]
    num_reviews = len(dff_sim)
    pmf_pos_reviews = (dff_sim['pmf_score'] > 4).sum() / num_reviews
    nps_pos_reviews = (dff_sim['nps_score'] > 8).sum() / num_reviews
    
    # lower thresholds for testing only
    result = (num_reviews >= 20) and (pmf_pos_reviews >= .5) and (nps_pos_reviews >= .5)
    
    return str(result)

def cant_live_without_superlative(rf6_app_uid):
    dff_sim = df_sim[df_sim['rf6_applicationUID'] == rf6_app_uid]
    num_reviews = len(dff_sim)
    pmf_pos_reviews = (dff_sim['pmf_score'] > 4).sum() / num_reviews
    
    # lower thresholds for testing only
    result = (num_reviews >= 20) and (pmf_pos_reviews >= .50)
    
    return str(result)

def ratings_distribution(rf6_app_uid):
    dff_sim = df_sim[df_sim['rf6_applicationUID'] == rf6_app_uid]
    dff_pos = dff_sim[dff_sim['pmf_score'] >= 3]
    dff_citizens = dff_sim[dff_sim.is_citizen == True]
    dff_delegates = dff_sim[dff_sim.is_top_delegate == True]
    return {
        'citizens': {
            'extremely_upset': (dff_citizens['pmf_score'] == 5).sum() / len(dff_citizens),
            'somewhat_upset': (dff_citizens['pmf_score'] == 4).sum() / len(dff_citizens),
            'neutral': (dff_citizens['pmf_score'] == 3).sum() / len(dff_citizens)
        },
        'top_delegates': {
            'extremely_upset': (dff_delegates['pmf_score'] == 5).sum() / len(dff_delegates),
            'somewhat_upset': (dff_delegates['pmf_score'] == 4).sum() / len(dff_delegates),
            'neutral': (dff_delegates['pmf_score'] == 3).sum() / len(dff_delegates)
        }
    }

def councils_distribution(rf6_app_uid):
    dff_sim = df_sim[df_sim['rf6_applicationUID'] == rf6_app_uid]
    result = {}
    for council in COUNCILS:
        dff_council = dff_sim[dff_sim['governance_membership'].apply(lambda x: council in x)]
        if dff_council.empty:
            continue
        result.update({
            council.replace(' ','_').lower(): {
                'count_attestations': len(dff_council),
                'avg_pmf_score': dff_council['pmf_score'].mean(),
                'avg_nps_score': dff_council['nps_score'].mean()
            }
        })
    return result

In [18]:
metrics = []
metrics.append({
    'name': 'count_total_attestations',
    'description': 'Number of Attestations',
    'data': df_sim.groupby('rf6_applicationUID')['uid'].count().to_dict()
})
metrics.append({
    'name': 'count_citizen_attestations',
    'description': 'Number of Attestations by Citizens',
    'data': df_sim[df_sim.is_citizen].groupby('rf6_applicationUID')['uid'].count().to_dict()
})
metrics.append({
    'name': 'count_delegate_attestations',
    'description': 'Number of Attestations by Top Delegates',
    'data': df_sim[df_sim.is_top_delegate].groupby('rf6_applicationUID')['uid'].count().to_dict()    
})
metrics.append({
    'name': 'avg_nps_score',
    'description': 'Average NPS score of Citizens and Top Delegates',
    'data': df_sim.groupby('rf6_applicationUID')['nps_score'].mean().to_dict()
})
metrics.append({
    'name': 'most_positive_superlative',
    'description': 'Most positive superlative (20 reviews, 95% high PMF and 95% high NPS)',
    'data': {uid: most_positive_superlative(uid) for uid in df_sim['rf6_applicationUID'].unique()}
})
metrics.append({
    'name': 'cant_live_without_superlative',
    'description': "Can't live without superlative (20 reviews, 90% high PMF)",
    'data': {uid: cant_live_without_superlative(uid) for uid in df_sim['rf6_applicationUID'].unique()}
})
metrics.append({
    'name': 'percentage_distributions',
    'description': "Percentage distribution of different ratings by citizens and top delegates",
    'data': {uid: ratings_distribution(uid) for uid in df_sim['rf6_applicationUID'].unique()}
})    
metrics.append({
    'name': 'elected_governance_reviews',
    'description': 'Reviews from elected governance members',
    'data': {uid: councils_distribution(uid) for uid in df_sim['rf6_applicationUID'].unique()}
})

In [19]:
with open("data/dummy_metrics.json", "w") as f:
    json.dump(metrics,f,indent=2)

In [21]:
df_sim.to_csv("data/normalized_attestations.csv")